In [1]:
from to_objects import get_objects
from rdflib import Graph

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [2]:
g = Graph()
g.parse('tutorial/bldg2.ttl')

<Graph identifier=N571587349fc447e3aab5aba453c75310 (<class 'rdflib.graph.Graph'>)>

In [3]:
zones = get_objects(g)

In [4]:
zone1 = zones[0]
zone1.spaces[0].area

Value(value=1500.0, unit='FT2')

In [8]:
zones[0].spaces

[Space(name='urn:hpflex/hpflex_demo#space_1_1', area=Value(value=1500.0, unit='FT2'))]

# Drafting

In [26]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from rdflib import Graph, Literal, Namespace, URIRef
from semantic_mpc_interface.namespaces import * 
from semantic_mpc_interface.utils import * 
from buildingmotif.ingresses import CSVIngress, TemplateIngress, Record

2025-07-25 22:23:46,075 | root |  CRITICAL: Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


In [11]:
bm = BuildingMOTIF("sqlite://")
model = Model.create(HPF)


In [12]:
tmp = Library.load(directory='dataclasses/templates/brick-templates')

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an identity for (<class 'buildingmotif.database.tables.DepsAssociation'>, (1,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  self.bm.session.flush()
/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an identity for (<class 'buildingmotif.database.tables.DepsAssociation'>, (2,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  self.bm.session.flush()
/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an ide

In [13]:
zone = tmp.get_template_by_name('hvac-zone')

In [14]:
zone_in = zone.inline_dependencies()

In [15]:
display(zone_in.all_parameters)

{'name',
 'space_name',
 'space_name-area_name',
 'space_name-area_unit',
 'space_name-area_value',
 'window_name',
 'window_name-area_name',
 'window_name-area_unit',
 'window_name-area_value',
 'window_name-azimuth_name',
 'window_name-azimuth_value',
 'window_name-tilt_name',
 'window_name-tilt_value'}

In [16]:
print(zone_in.body.serialize(format = 'ttl'))

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <http://qudt.org/schema/qudt/> .

<urn:___param___#name> a brick:HVAC_Zone ;
    brick:hasPart <urn:___param___#space_name>,
        <urn:___param___#window_name> .

<urn:___param___#space_name> a brick:Space ;
    brick:area <urn:___param___#space_name-area_name> .

<urn:___param___#space_name-area_name> ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#space_name-area_unit> ;
    brick:value <urn:___param___#space_name-area_value> .

<urn:___param___#window_name> a brick:Window ;
    brick:area <urn:___param___#window_name-area_name> ;
    brick:azimuth <urn:___param___#window_name-azimuth_name> ;
    brick:tilt <urn:___param___#window_name-tilt_name> .

<urn:___param___#window_name-area_name> ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#window_name-area_unit> ;
    brick:value <urn:___param___#window_name-area_value>

In [17]:
def get_var_name(graph, node):
    pre, ns, local = graph.compute_qname(node)
    if PARAM == ns:
        q_n = f"?{local}".replace('-','_')
    else:
        q_n = convert_to_prefixed(node, graph).replace('-','_')
    return q_n

In [18]:
def make_where(graph):
    where = []
    for s,p,o in graph.triples((None,None,None)):
        # print(s,p,o)
        qs = get_var_name(graph, s)
        qo = get_var_name(graph, o)
        qp = convert_to_prefixed(p, graph).replace('-','_')
        where.append(f"{qs} {qp} {qo} .")
    return "\n".join(where)

In [19]:
def get_query(graph):
     where = make_where(graph)
     prefixes = get_prefixes(graph)
     query = f"""{prefixes}\nSELECT DISTINCT * WHERE {{ {where} }}"""
     return query

In [20]:
query = get_query(zone_in.body)

In [21]:
print(query)

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

In [22]:
g = Graph()
g.parse('tutorial/bldg2.ttl')

<Graph identifier=Ncf59473816a24aa78a2e4428ac3db622 (<class 'rdflib.graph.Graph'>)>

In [23]:
df = query_to_df(query, g)

In [24]:
class Value:
    unit: str
    value: float

In [25]:
df

,window_name_azimuth_value,name,window_name_area_unit,space_name_area_name,window_name_azimuth_name,window_name_tilt_name,window_name_area_value,space_name_area_unit,space_name_area_value,window_name,space_name,window_name_tilt_value,window_name_area_name
0,180.0,urn:hpflex/hpflex_demo#zone_1,unit:FT2,urn:hpflex/hpflex_demo#space_1_1_area,urn:hpflex/hpflex_demo#window_1_1_azimuth,urn:hpflex/hpflex_demo#window_1_1_tilt,150.0,unit:FT2,1500.0,urn:hpflex/hpflex_demo#window_1_1,urn:hpflex/hpflex_demo#space_1_1,90.0,urn:hpflex/hpflex_demo#window_1_1_area
1,180.0,urn:hpflex/hpflex_demo#zone_2,unit:FT2,urn:hpflex/hpflex_demo#space_2_1_area,urn:hpflex/hpflex_demo#window_2_1_azimuth,urn:hpflex/hpflex_demo#window_2_1_tilt,150.0,unit:FT2,1500.0,urn:hpflex/hpflex_demo#window_2_1,urn:hpflex/hpflex_demo#space_2_1,90.0,urn:hpflex/hpflex_demo#window_2_1_area
2,180.0,urn:hpflex/hpflex_demo#zone_3,unit:FT2,urn:hpflex/hpflex_demo#space_3_1_area,urn:hpflex/hpflex_demo#window_3_1_azimuth,urn:hpflex/hpflex_demo#window_3_1_tilt,150.0,unit:FT2,1500.0,urn:hpflex/hpflex_demo#window_3_1,urn:hpflex/hpflex_demo#space_3_1,90.0,urn:hpflex/hpflex_demo#window_3_1_area
3,180.0,urn:hpflex/hpflex_demo#zone_4,unit:FT2,urn:hpflex/hpflex_demo#space_4_1_area,urn:hpflex/hpflex_demo#window_4_1_azimuth,urn:hpflex/hpflex_demo#window_4_1_tilt,150.0,unit:FT2,1500.0,urn:hpflex/hpflex_demo#window_4_1,urn:hpflex/hpflex_demo#space_4_1,90.0,urn:hpflex/hpflex_demo#window_4_1_area


In [ ]:
zone_in.generate_csv('test.csv')

{'name': 'name',
 'window_name-tilt_name': 'window_name-tilt_name',
 'window_name-area_name': 'window_name-area_name',
 'space_name-area_name': 'space_name-area_name',
 'window_name': 'window_name',
 'space_name': 'space_name',
 'window_name-azimuth_name': 'window_name-azimuth_name'}

In [40]:
zone_part = zone_in.evaluate({p:HPF[p] for p in zone_in.all_parameters if p.split('_')[-1] == 'name'})

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:420: UserWarning: Parameters "window_name-azimuth_value, space_name-area_unit, window_name-tilt_value, window_name-area_value, window_name-area_unit, space_name-area_value" were not provided during evaluation
  warnings.warn(


In [52]:
zone_part.generate_csv('test.csv')
# dependencies are showing up in headers for some reason - can just remove these. 
dependencies = zone_part.get_dependencies()

df = pd.read_csv('test.csv')

remove_cols = [dependency.template.name for dependency in zone.get_dependencies()]
df = df.drop(columns=remove_cols)

# Save the new CSV
df.to_csv('test.csv', index=False)
